## Import modules

In [ ]:
%matplotlib ipympl

import matplotlib.pyplot as plt
import numpy as np
import obspy
import pandas as pd

## Define the picking function with option to plot waveforms with picks

In [ ]:
def pick(wf_file, md_file, recordlength=1.5, plot=False):
    df_sta = pd.read_csv(md_file, delim_whitespace=True, header=None, names=['sta', 'net', 'x', '_y', '?', '??'])
    df_sta['y'] = 3e3 - df_sta['_y']
    
    df = pd.DataFrame()
    st = obspy.read(wf_file)
    for idx in range(len(st)):
        tr = st[idx]
        tr.stats.network  = 'AA'
        tr.stats.station  = df_sta.loc[idx, 'sta']
        tr.stats.channel  = 'DPZ'
        tr.stats.distance = df_sta.loc[idx, 'x']
        tr.stats.t0_first = np.nonzero(tr.data > np.median(np.abs(tr)) * 5)[0][0] * tr.stats.delta
        tr.stats.t0_max   = np.argmax(np.abs(tr)) * tr.stats.delta
        df = df.append(
            pd.DataFrame(
                [[tr.stats.station, tr.stats.t0_first, tr.stats.t0_max]],
                columns=['sta', 't0_first', 't0_max']
            ),
            ignore_index=True
        )

    if plot is True:
        fig = st.plot(
            type='section', 
            time_down=True,
            recordlength=recordlength,
            fillcolors=('k', 'w'), 
            handle=True
        )

        ax = fig.axes[0]
        xmin, xmax = ax.get_xlim()
        delta_x    = xmax - xmin

        pick_marker_half_width = 1/(2.5*len(st))
        for tr in st:
            ax.axhline(
                y=tr.stats.t0_first,
                xmin=(tr.stats.distance * 1e-3 - xmin)/delta_x-pick_marker_half_width,
                xmax=(tr.stats.distance * 1e-3 - xmin)/delta_x+pick_marker_half_width,
                color='r'
            )
            ax.axhline(
                y=tr.stats.t0_max,
                xmin=(tr.stats.distance * 1e-3 - xmin)/delta_x-pick_marker_half_width,
                xmax=(tr.stats.distance * 1e-3 - xmin)/delta_x+pick_marker_half_width,
                color='b'
            )
    return (df)

## Pick times are return as a pandas.DataFrame

In [ ]:
pick('../data/00012.su', '../data/STATIONS')

## Plot waveforms with picks

In [ ]:
df = pick('../data/00023.su', '../data/STATIONS', plot=True)